In [1]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing required packages for this notebook session
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("descartes")
sc.install_pypi_package("shapely")
sc.install_pypi_package("pyproj==2.2.2")
sc.install_pypi_package("geopandas")
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1609430674348_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/30/f2/10c822cb0ca5ebec58bd1892187bc3e3db64a867ac26531c6204663fc218/matplotlib-3.3.3-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/af/fa/c1302a26d5e1a17fa8e10e43417b6cf038b0648c4b79fcf2302a4a0c5d30/Pillow-8.0.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d2/46/231de802ade4225b76b96cffe419cf3ce52bbe92e3b092cf12db7d11c207/kiwisolver-1.3.1-cp37-cp37m-manylinux1_x86_64.whl

  Usin

In [4]:
traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_15101/")
# selecting traces which are far more than 40km from the nearest shape
traces.filter("min_distance > 40000").show(50,False)
# Results --> id_avl --> 11773, line_id --> 1047

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------+-------+----------+----------+--------+------------------+-------------+--------------+---------+---------+--------+--------------+--------+------------------+------------------+-------------------+-------------------+
|dt_avl                 |id_avl|line_id|trace_y   |trace_x   |hour_avl|hour_diff         |region       |time_variation|trip_id  |direction|route_id|trip_head     |shape_id|min_distance      |min_shape_sequence|min_shape_coord_lat|min_shape_coord_lon|
+-----------------------+------+-------+----------+----------+--------+------------------+-------------+--------------+---------+---------+--------+--------------+--------+------------------+------------------+-------------------+-------------------+
|2015-10-01 07:38:03.000|11773 |1047   |-23.859152|-46.71662 |7       |106.44300079345703|PARELHEIROS  |80.0          |3751-10-0|0        |3751-10 |Metrô Itaquera|61772   |43173.726409340736|356.0             |-23.542494         |-46.471243       

In [29]:
from pyspark.sql.functions import f
traces.filter((col("id_avl") == 11773) & (col("line_id") == 1047)) \
    .select("dt_avl","min_shape_sequence","trace_y","trace_x") \
    .sort("dt_avl") \
    .repartition(1) \
    .write.csv("s3://mobility-traces-sp/tests-map-views/2-map-matchin-view/MO_15101_id_avl_11773_line_1047/")
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
traces.select("line_id").distinct().show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|line_id|
+-------+
|  35148|
|    474|
|     65|
|   1224|
|    558|
|    418|
|   2184|
|   1175|
|    243|
|  35216|
+-------+
only showing top 10 rows

In [32]:
traces.filter("line_id == 35148").select("id_avl").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
|id_avl|
+------+
| 10918|
| 11070|
| 11187|
| 10822|
| 11207|
| 11205|
| 47150|
| 10835|
+------+

In [42]:
traces.filter((col("line_id") == 35148) & (col("id_avl") == 10835)).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

18

In [54]:
from pyspark.sql import functions as f
df = traces.groupBy("line_id","id_avl").agg(f.count('id_avl').alias("count_traces"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
df.filter((col("line_id") == 35148) & (col("id_avl") == 10835)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------------+
|line_id|id_avl|count_traces|
+-------+------+------------+
|  35148| 10835|          18|
+-------+------+------------+

In [65]:
df.select(f.mean("count_traces"),f.min("count_traces"),f.max("count_traces"),f.stddev("count_traces")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+-----------------+-------------------------+
|avg(count_traces)|min(count_traces)|max(count_traces)|stddev_samp(count_traces)|
+-----------------+-----------------+-----------------+-------------------------+
|571.3859121388732|                1|            27060|       487.96158812725486|
+-----------------+-----------------+-----------------+-------------------------+

In [63]:
df.filter("count_traces == 27060").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------------+
|line_id|id_avl|count_traces|
+-------+------+------------+
|  32932| 36669|       27060|
+-------+------+------------+

In [64]:
# There is the buses with few registers 1, average  is 570 register, the max with 27060

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[line_id: bigint, id_avl: bigint, count_traces: bigint]

In [68]:
traces.filter((f.col("id_avl") == 36669) & (f.col("line_id") == 32932)).sort("dt_avl") \
    .select("dt_avl","min_shape_sequence").show(1500,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+------------------+
|dt_avl                 |min_shape_sequence|
+-----------------------+------------------+
|2015-10-01 07:22:49.000|436.0             |
|2015-10-01 07:22:50.000|436.0             |
|2015-10-01 07:22:51.000|436.0             |
|2015-10-01 07:22:53.000|436.0             |
|2015-10-01 07:22:54.000|436.0             |
|2015-10-01 07:22:55.000|436.0             |
|2015-10-01 07:22:56.000|436.0             |
|2015-10-01 07:22:58.000|436.0             |
|2015-10-01 07:23:00.000|436.0             |
|2015-10-01 07:23:01.000|436.0             |
|2015-10-01 07:23:02.000|436.0             |
|2015-10-01 07:23:04.000|436.0             |
|2015-10-01 07:23:05.000|436.0             |
|2015-10-01 07:23:06.000|436.0             |
|2015-10-01 07:23:07.000|436.0             |
|2015-10-01 07:23:08.000|436.0             |
|2015-10-01 07:23:10.000|436.0             |
|2015-10-01 07:23:13.000|436.0             |
|2015-10-01 07:23:14.000|436.0             |
|2015-10-0

In [ ]:
# There is buses starting by the end of the shape

In [74]:
df2 = traces.groupBy("id_avl","line_id").agg(f.countDistinct("min_shape_sequence").alias("shape_seq_count"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df2.select(f.mean("shape_seq_count"),f.min("shape_seq_count"),f.max("shape_seq_count"),f.stddev("shape_seq_count")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+----------------------------+
|avg(shape_seq_count)|min(shape_seq_count)|max(shape_seq_count)|stddev_samp(shape_seq_count)|
+--------------------+--------------------+--------------------+----------------------------+
|  150.43614087861127|                   1|                 664|           80.82167888005603|
+--------------------+--------------------+--------------------+----------------------------+

In [83]:
# Results there is buses that has few shapes matching 
# what means that even with thousand of traces the bus does not move like the bus with id_vl 36669 and line_id 3932

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
traces.filter((f.col("id_avl") == 36669) & (f.col("line_id") == 32932)) \
    .sort("dt_avl") \
    .repartition(1) \
    .write.csv("s3://mobility-traces-sp/tests-map-views/2-map-matchin-view/MO_15101_id_avl_36669_line_32932/") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
traces.filter((f.col("id_avl") == 36669) & (f.col("line_id") == 32932)).select("min_shape_sequence").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|min_shape_sequence|
+------------------+
|             438.0|
|             439.0|
|             435.0|
|             236.0|
|             437.0|
|             436.0|
+------------------+

In [84]:
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+---------------+
|id_avl|line_id|shape_seq_count|
+------+-------+---------------+
| 50124|   1132|            217|
| 37021|  34490|            107|
| 40127|  33992|            160|
| 58180|   1123|            199|
| 58209|   2198|            261|
| 11801|  33191|            166|
| 57416|   1131|            212|
| 39263|  33916|            136|
| 38721|   1993|            194|
| 14084|      1|            183|
| 40073|   1733|             53|
| 51950|  33516|            330|
| 11351|   1472|              1|
| 56862|  33865|            129|
| 55725|    947|            106|
| 14064|    406|             75|
| 12659|  33299|            228|
| 12762|  34134|             58|
| 12458|  33191|              2|
| 35521|    402|              3|
+------+-------+---------------+
only showing top 20 rows

In [85]:
# Trying to analyze a bus with more than 200 shapes --> id_avl 50124, line_id 1132

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
traces.filter((f.col("id_avl") == 50124) & (f.col("line_id") == 1132)).sort("dt_avl").repartition(1) \
    .write.csv("s3://mobility-traces-sp/tests-map-views/2-map-matchin-view/MO_15101_id_avl_50124_line_1132/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Results
- There is traces far away from the nearest shape by km (like 40km)
- There is bus with few registers in one day
- There is bus with thousand of traces but only with few shape (meaning that the bus did not move by city)
- By watching the file proceduced by the last test of the bus 50124 line 1132, I watched that the shape follows a sequence (ascending), but it did  start by shape sequence 1, started in shape 6 and in other trips started in other shapes

In [2]:
sc.install_pypi_package("haversine")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
from haversine import haversine, Unit
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

def calculate_distace(lon1,lat1,lon2,lat2,t):

    if lon2 and lat2:
        coord_1 = float(lon1),float(lat1)
        coord_2 = float(lon2),float(lat2)
        distance = haversine(coord_1,coord_2,unit=Unit.METERS)
        
        return (distance/float(t)) * 3.6
    else:
        return -1



get_distance_udf = udf(calculate_distace, FloatType())

window = Window.partitionBy("id_avl","line_id").orderBy('dt_avl')

df333 = traces_mo.select("*", get_distance_udf(F.col("trace_x"),F.col("trace_y"),F.lag(col("min_shape_coord_lon")).over(window),F.lag(col("min_shape_coord_lat")).over(window),F.col("time_variation")).alias("maie"))




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
df333.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+----------+----------+--------+------------------+-----------+--------------+---------+---------+--------+---------+--------+------------------+------------------+-------------------+-------------------+----------+
|              dt_avl|id_avl|line_id|   trace_y|   trace_x|hour_avl|         hour_diff|     region|time_variation|  trip_id|direction|route_id|trip_head|shape_id|      min_distance|min_shape_sequence|min_shape_coord_lat|min_shape_coord_lon|      maie|
+--------------------+------+-------+----------+----------+--------+------------------+-----------+--------------+---------+---------+--------+---------+--------+------------------+------------------+-------------------+-------------------+----------+
|2015-10-01 06:56:...|  8577|  35033|-23.520062|-46.701605|       6|1.0399999618530273|       LAPA|          45.0|211L-10-1|        1| 211L-10| Mandaqui|   57855|  4.88280389883034|               3.0|         -23.520024|         -46.701629|    

In [1]:
# Getting statistic of the number of shapes per vehicle in a day and of traces per vehicle

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1609430674348_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pyspark.sql.functions as f

# files' statistics header
csv_out_shape_count = "day,total_size,shape_seq_count_mean,shape_seq_count_min,shape_seq_count_max,shape_seq_count_stddev,shape_seq_count_quantile_25,shape_seq_count_quantile_50,shape_seq_count_quantile_75,shape_seq_count_below_2,shape_seq_count_below_5\n"
csv_out_trace_count = "day,total_size,traces_count_mean,traces_count_min,traces_count_max,traces_count_stddev,traces_count_quantile_25,traces_count_quantile_50,traces_count_quantile_75,traces_count_below_2,traces_count_below_10\n"


for day in range(1,32):
    
    # reading map matching files
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510"+ str(day) + "/")
    
    # getting the number of registers
    total_size = traces.count()


    # counting the number of different shape sequences for a bus in a day
    shape_count = traces.groupBy("id_avl","line_id").agg(f.countDistinct("min_shape_sequence").alias("shape_seq_count"))
    # getting some statistics
    stats_shape_seq_count = shape_count.agg(f.mean('shape_seq_count').alias('mean'),
                       f.min('shape_seq_count').alias('min'),
                       f.max('shape_seq_count').alias('max'),
                       f.stddev('shape_seq_count').alias("stddev")).collect()
    
    # getting the quantiles
    shape_seq_count_quantile = shape_count.approxQuantile("shape_seq_count", [0.25,0.5,0.75], 0.0001)
    
    # selecting traces below 2 shapes in the traces --> it means the bus did not moved 
    shape_below_2 = shape_count.filter("shape_seq_count < 2").count()
    # selecting traces below 5 shapes in the traces --> it means the bus did not moved 
    shape_below_5 = shape_count.filter("shape_seq_count < 5").count()
    
    csv_out_shape_count += "MO_1510" + str(day) +","+ str(total_size) +","+ str(stats_shape_seq_count[0]["mean"]) +","+ str(stats_shape_seq_count[0]["min"]) + "," + str(stats_shape_seq_count[0]["max"]) + "," + str(stats_shape_seq_count[0]["stddev"]) + "," +str(shape_seq_count_quantile[0]) + "," +str(shape_seq_count_quantile[1]) +"," +str(shape_seq_count_quantile[2]) +","+ str(shape_below_2) + ","+ str(shape_below_5) + "\n"



    # counting the number of traces for a bus in a day
    trace_count = traces.groupBy("id_avl","line_id").agg(f.count("id_avl").alias("traces_count_per_bus"))
    # getting some statistics
    stats_trace_count = trace_count.agg(f.mean('traces_count_per_bus').alias('mean'),
                       f.min('traces_count_per_bus').alias('min'),
                       f.max('traces_count_per_bus').alias('max'),
                       f.stddev('traces_count_per_bus').alias("stddev")).collect()
    # getting the quantiles
    trace_count_quantile = trace_count.approxQuantile("traces_count_per_bus", [0.25,0.5,0.75], 0.0001)

    # selecting traces below 2 traces
    traces_below_2 = trace_count.filter("traces_count_per_bus < 2").count()
    
    # selecting traces below 10 traces
    traces_below_10 = trace_count.filter("traces_count_per_bus < 10").count()
    
    csv_out_trace_count += "MO_1510" + str(day) +","+ str(total_size) +","+ str(stats_trace_count[0]["mean"]) +","+ str(stats_trace_count[0]["min"]) + "," + str(stats_trace_count[0]["max"]) + "," + str(stats_trace_count[0]["stddev"]) + "," +str(trace_count_quantile[0]) + "," +str(trace_count_quantile[1]) +"," +str(trace_count_quantile[2]) +","+ str(traces_below_2) + ","+ str(traces_below_10) + "\n"

import boto3   
s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out_shape_count,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/6-shapes_seq_per_bus.csv')    
s3.put_object(Body=bytes(csv_out_trace_count,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/7-traces_per_bus.csv')    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'D45A435F71465B5E', 'HostId': 'W73X8HFn4ItlIgTtUOSloDHSZghMBIh+qGqqFFbSD2YAP6JwUZjosWMZ3I7CR1E7hSJ/yfqKujc=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'W73X8HFn4ItlIgTtUOSloDHSZghMBIh+qGqqFFbSD2YAP6JwUZjosWMZ3I7CR1E7hSJ/yfqKujc=', 'x-amz-request-id': 'D45A435F71465B5E', 'date': 'Thu, 31 Dec 2020 20:00:56 GMT', 'etag': '"f7d8e106b5ed8460bab5e0ce5a262069"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"f7d8e106b5ed8460bab5e0ce5a262069"'}

# Results 
## Number of shapes per bus
- There are buses with only one shape matched, and 678 shapes max
- The mean of matched shape is between 98 and 153
- First quantile 25% - there are value between 1 - 101. We find out there is 1 shape for buses in weekends
- second quantile 50% - 97 and 160 shapes, 
- third quantile 75% - 178 and 211 shapes
- there are between 1500 and 5900 buses with 2 or less shapes matched with their traces. 
- there are between 2200 and 8000 buses witn only 5 or less shapes matched with their traces.

## Number of traces per bus
- There are buses with only 1 traces per day
- At average for max values, there are buses with over tahn 30000 traces in one day
- Average between 493 and 776 traces per bus during a day
- There are between 43 and 194 buses with 2 or less traces in a day
- There are between 263 and 669 buses with 10 or less traces in a day

## Next steps
- understand the first quantile (25%) of number of shapes and traces per bus